In [167]:
import warnings
warnings.filterwarnings("ignore")

In [168]:
from langchain_community.chat_models import GigaChat
from langchain_core.messages import SystemMessage 
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain


import requests
from bs4 import BeautifulSoup
import uuid

Задача:

Необходимо разработать умного ассистента, который выполнит следующие шаги:

1. Поиск информации в интернете:
- Ассистент должен находить в интернете последние новости, связанные с целевой аудиторией - мамами. Основной фокус на новости, касающиеся важности раннего обучения детей арифметике, скорочтению и другим предметам.

2. Создание текста для блога:
- На основе найденной новости ассистент должен создать краткий, но содержательный текст, подходящий для блога. Текст должен быть написан простым и понятным языком, рассчитанным на аудиторию мам.

3. Создание текста для изображения:
- Из текста для блога нужно выделить основную мысль или лозунг, который будет использоваться для создания изображения (например, баннера или инфографики). Текст должен быть кратким и запоминающимся.

4. Создание изображения:
- На основе подготовленного текста для изображения необходимо сгенерировать визуальный контент (изображение). Это может быть простая графика, инфографика или баннер.

5. Сохранение результатов:
- Ассистент должен сохранить следующие файлы:
- Документ с текстом для блога в формате .docx.
- Сгенерированное изображение в формате .png или .jpg.
- Исходный текст для изображения, который использовался при создании графики, в отдельном текстовом файле.

Требования:

1. Ассистент должен быть автономным и автоматически выполнять указанные шаги, начиная с поиска информации и заканчивая сохранением результатов.
2. В процессе разработки ассистента необходимо использовать технологию ChatGPT (или аналоги) для генерации текстов на этапах 2 и 3.
3. Необходимо предоставить отчет о проделанной работе, где будут описаны:
- Используемые технологии и библиотеки.
- Подходы к решению каждой из задач.
- Возможные улучшения и оптимизации ассистента.

Ожидаемый результат:

- Запуск умного ассистента, который выполнит все вышеописанные шаги.
- Предоставление всех созданных файлов (текст для блога, изображение, исходный текст для изображения).
- Отчет о проделанной работе.

Критерии оценки:

- Корректность выполнения каждого этапа задачи.
- Качество созданного текста и изображения.
- Автономность и стабильность работы ассистента.
- Понимание применяемых технологий и обоснованность их выбора.
- Креативность в подходе к решению задачи.

Дополнительных критериев нет, выполните работу по заданным параметрам, как считаете нужным.

## 1.1 Первый подход поиска новостей

In [188]:
def fetch_latest_news(keywords):
    url = f"https://www.babyblog.ru/search?request={'+'.join(keywords)}"
    print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('div', class_='post-body-content-info')[:25]
    news = []
    for article in articles:
        summary = article.find('p').text
        link = article.find('a')['href']
        news.append({'summary': summary, 'link': f'https://www.babyblog.ru{link}'})
    return news

In [189]:
#keywords = ['раннее обучение', 'дети', 'арифметика', 'скорочтение']
keywords = ['дети', 'обучение']

In [190]:
news_list = fetch_latest_news(keywords)

https://www.babyblog.ru/search?request=дети+обучение


In [191]:
news_list

[{'summary': 'Хочу все же добиться. В прошлом году мне мямлили, слили можно сказать. Но школа предложила только надомное.',
  'link': 'https://www.babyblog.ru/user/rkrjdrf/3213035'},
 {'summary': 'Здравствуйте,хотела бы спросить,как ваши дети с ВПС посещают школу,на каких формах образовательного процесса они находятся.?',
  'link': 'https://www.babyblog.ru/community/Zdorovjebaby/post/3176471'},
 {'summary': 'Ниже 7 способов, как усваивать информацию через игру:\n',
  'link': 'https://www.babyblog.ru/user/ibulycheva/151943'},
 {'summary': '',
  'link': 'https://www.babyblog.ru/community/undefined/post/717267'},
 {'summary': 'Девочки,поделитесь своими мыслями,опытом и историями,как вы отпустили свою кровинушку,выпускника(цу) школы получать высшее образование за пределами страны?Не побоялись?Оправдались ли все ваши ожидания от поездки?Как изменился ребенок после возвращения домой?',
  'link': 'https://www.babyblog.ru/questions/petrova19882020/45623'},
 {'summary': 'Я тут вспомнила что ког

## 1.2 Второй подход поиска новостей

In [183]:
def fetch_latest_news(api_key, query, language='ru', page_size=100):
    url = 'https://newsapi.org/v2/everything'
    params = {
        'q': query,
        'language': language,
        'pageSize': page_size,
        'sortBy': 'publishedAt',
        'apiKey': api_key
    }
    response = requests.get(url, params=params)
    data = response.json()
    if data['status'] == 'ok':
        return data['articles']
    else:
        raise Exception(f"Error fetching news: {data.get('message')}")

In [224]:
NEWS_API_KEY = '4ffadf49a5db40828d186ba16a395565'
QUERY = 'развитие детей OR детское обучение OR раннее обучение OR \
скорочтение OR арифметика OR образование детей'

In [225]:
news_articles = fetch_latest_news(NEWS_API_KEY, QUERY)

In [226]:
news_articles

[{'source': {'id': None, 'name': 'Tass.ru'},
  'author': None,
  'title': 'Херсонская область присоединилась к медиапроекту "Герой будущего времени"',
  'description': 'Проект направлен на патриотическое, нравственное, психологическое и духовное развитие детей и молодежи',
  'url': 'https://tass.ru/obschestvo/22196671',
  'urlToImage': 'https://cdn-storage-media.tass.ru/thumb/ru/obschestvo/22196671?u=2024-10-23T08:11:35.591526',
  'publishedAt': '2024-10-23T08:11:35Z',
  'content': ', 23 . //. " ", , , . , .\r\n" . ? , , , , , . &lt;...&gt; , , , , , ", - .\r\n , , .\r\n " . ", , . , , , , - -, " - ".'},
 {'source': {'id': None, 'name': 'Vz.ru'},
  'author': None,
  'title': 'Замглавы Геленджика: Выступление детского ансамбля на кладбище было актом памяти педагога',
  'description': 'Замглавы Геленджика по социальным вопросам Янина Скорикова заявила, что информация о выступлении детского ансамбля на кладбище была искажена. Концерт был дан в память о бывшем преподавателе школы искусств,

## 2.Построение текста для блога

Был выбран первый подход для поиска новостей

### Получение ключа для Gigachat 

In [192]:
auth = 'YmE5ZTA1ZTAtOTI0Ny00Y2RkLTk5OGYtNDU4NWQ4NzEyMTE5OjZiYjc4MjY1LTMzZWEtNGE5OC05ODg4LTRiNjQ4M2EyNGFjMQ=='

In [193]:
def get_token(auth_token, scope='GIGACHAT_API_PERS'):
    """
      Выполняет POST-запрос к эндпоинту, который выдает токен.

      Параметры:
      - auth_token (str): токен авторизации, необходимый для запроса.
      - область (str): область действия запроса API. По умолчанию — «GIGACHAT_API_PERS».

      Возвращает:
      - ответ API, где токен и срок его "годности".
      """
    rq_uid = str(uuid.uuid4())

    # API URL
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth}'
    }

    payload = {
        'scope': scope
    }

    try:
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return -1

     

In [194]:
response = get_token(auth)
if response != 1:
  print(response.text)
  giga_token = response.json()['access_token']

{"access_token":"eyJjdHkiOiJqd3QiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwiYWxnIjoiUlNBLU9BRVAtMjU2In0.nUHOfTqdNFUkKdAuYP5SPspt4XuZqqObyVcFbLeKpGiDT3jWYKTSJC-LYX6UewKofrM5S_L18xV0mQOim2ZS4tq68CbKqh_ovvWLttbcXezBELxc9Xy80QZEtHS9fokYJlYlFtBgVZPddD9quklYbByBlLyP4woMLVR7cT8JPWR9ppweP7ROS0Ljo4uauQyh7Apr5JEG0t439CTUYR7yXALzJajvh_YK1dxOfVQpQKH1sbKpt_pP9wT8p7ofPKDqb6BCaOm6TYwtM8FWa46_zY_95L938w27c9OICxgs5sdNGQFMef8RBrx4yycL2sOhAlTXW4_sDqUWj4WAxGSlaw._IAor-8rKGqwPioIoMaycA.twlng2XLKwuLafkIhQOA0DPw0yLN8nGq-IprjWXVZvkqXFA0_cz4NewB4nI4TFSNpJ3t2KtWc-hJFPypmlvzOR21En2ZxNf6964j7SEaqTkduSWC7JBbGDF1LpJbZ5spp84QOfbS500aosAVGg5YyRXkKfIobemccq1RIGcFJB0xmhh8Gm4BvqgMf0Gclkr8Higrp_lDAeIHinzkHDOvYEuxzaEUfzM-9qmVYiN5xjgV1Hgjfl7_PJ2AF7N70DZBPfe5s47sOnzi7_1cXd6G_AmQDDmsnaJFolzPD0g2S_qxLO0I7pmE3_17ZTZkNVw3GUAFDOpet8QHBnneIeAMLiP9QN5qlCk5p6z_xViEUQQf6akjmq3R13K8qO-jDkijqRUjLkY_FQ3s51VTlJ-FtMz64aFWOXwjXJNHLaeqH-LpSO30xd0-yCVTYGHyPZTWU2wHpKIiHDZ3Oz12oD2Bi9bca7RSOAf3uuahsvT5fTkB2ZBf6Mr8cbbfkOCDHgZcWA_i4rWWraIqiGhcacB-I2lqDGK6e

### Генерация ответов

In [205]:
chat = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_PERS", access_token = giga_token)

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are an AI assistant who shares everything you know.\
            Based on the text of the article you received, you have to write a\
            text that is written in simple and clear language, aimed at an audience\
            of moms.You have to pretend you don't know the text of the article,\
            i.e. don't respond to the article itself, but create your own,\
            based on the one available. Talks in Russian. Each response must be at least 200 words."
        ),  
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),  
    ]
)

chad = LLMChain(
    llm=chat,
    prompt=prompt,
)

In [211]:
def chat_with_memory(user_input):
    response = chad.predict(human_input=user_input)
    print(f'Оригинальный текст: {user_input} \n\nСгенерированный текст: {response} \n\n')
    return response

generated_texts = []

user_input = news_list[6]['summary']
if user_input:
    ai_respon = chat_with_memory(user_input)
    generated_texts.append(ai_respon)

Оригинальный текст: Некоторые взрослые люди, которые не изучали французский, немецкий или английский в школе, уверены, что записываться на курсы уже нет смысла. Начинать постигать иностранный язык в действительности вполне можно практически в хоть каком возрасте - ведь никогда не поздно накапливать знания. Но все-таки чем раньше - тем лучше: таким образом в запасе 

Сгенерированный текст: Учить иностранные языки никогда не поздно, но лучше начинать как можно раньше. 




## 3. Создание текста для изображения

In [212]:
chat = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_PERS", access_token = giga_token,)

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="Из текста для блога нужно извлечь главную мысль\
            или слоган.\
            Текст должен быть кратким и запоминающимся. Свяжите эту главную идею с детской темой.\
            Старайтесь использовать такие слова, как «из детства» и подобные, свойственные детям. Пишите на русском языке"
        ),  
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),  
    ]
)

chad = LLMChain(
    llm=chat,
    prompt=prompt,
)

In [213]:
def chat_with_memory(user_input):
    response = chad.predict(human_input=user_input)
    print(f'Оригинальный текст: {user_input} \n\n\nСгенерированный текст: {response} \n\n\n\n')
    return response

mottoes = []
for text in generated_texts:
    if text:
        ai_respon = chat_with_memory(text)
        mottoes.append(ai_respon)

Оригинальный текст: Учить иностранные языки никогда не поздно, но лучше начинать как можно раньше. 


Сгенерированный текст: "Из детства - с любовью к языкам!" 






## 4. Создание изображения

### 4.1 Первый подход к генерации изображений (недоделанный и очень долгий)

In [75]:
#!pip install diffusers transformers accelerate

   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 MB 660.6 kB/s eta 0:00:05
    --------------------------------------- 0.1/2.9 MB 465.5 kB/s eta 0:00:07
   - -------------------------------------- 0.1/2.9 MB 798.9 kB/s eta 0:00:04
   -- ------------------------------------- 0.2/2.9 MB 833.5 kB/s eta 0:00:04
   -- ------------------------------------- 0.2/2.9 MB 807.1 kB/s eta 0:00:04
   -- ------------------------------------- 0.2/2.9 MB 807.1 kB/s eta 0:00:04
   -- ------------------------------------- 0.2/2.9 MB 807.1 kB/s eta 0:00:04
   -- ------------------------------------- 0.2/2.9 MB 807.1 kB/s eta 0:00:04
   -- ------------------------------------- 0.2/2.9 MB 807.1 kB/s eta 0:00:04
   -- ------------------------------------- 0.2/2.9 MB 807.1 kB/s eta 0:00:04
   --- ------------------------------------ 0.3/2.9 MB 516.0 kB/s eta 0:00:06
   -

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\moldo\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\moldo\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\moldo\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\moldo\anaconda3\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\moldo\anaconda3\Lib\http\client.py", line 479, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "C:\Users\moldo\anaconda3\Lib\socket.py", line 707, in readinto
    return self._sock.recv_into(b)
           ^

   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
    --------------------------------------- 0.0/2.9 MB 245.8 kB/s eta 0:00:12
    --------------------------------------- 0.0/2.9 MB 245.8 kB/s eta 0:00:12
    --------------------------------------- 0.0/2.9 MB 245.8 kB/s eta 0:00:12
    --------------------------------------- 0.0/2.9 MB 245.8 kB/s eta 0:00:12
    --------------------------------------- 0.0/2.9 MB 245.8 kB/s eta 0:00:12
    --------------------------------------- 0.0/2.9 MB 245

In [15]:
from diffusers import StableDiffusionPipeline
import torch

def generate_image_from_text(prompt_text: str):

    model_id = "runwayml/stable-diffusion-v1-5"  # можно заменить на "stabilityai/stable-diffusion-2"
    pipe = StableDiffusionPipeline.from_pretrained(model_id)
    

    pipe = pipe.to("cuda")
    

    torch.backends.cudnn.benchmark = True
    

    image = pipe(prompt_text).images[0]
    print('Image generated')
    return image  


prompt_text = "An inspiring scene showing perseverance and strength, a journey towards a glowing distant goal."
image = generate_image_from_text(prompt_text)


image.save("1.png")
print("Image saved as generated_image.png")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Exception ignored in: <function tqdm.__del__ at 0x000001F5E6391E40>
Traceback (most recent call last):
  File "C:\Users\moldo\anaconda3\Lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "C:\Users\moldo\anaconda3\Lib\site-packages\tqdm\notebook.py", line 272, in close
    def close(self):

KeyboardInterrupt: 

KeyboardInterrupt



In [91]:
mottoes[0]

'"Не теряй веру в себя и свои силы, даже если путь к цели будет немного другим."'

### 4.2 Второй подход (выбранный)

In [22]:
#!pip install deep-translator

   ---------------------------------------- 0.0/42.3 kB ? eta -:--:--
   --------- ------------------------------ 10.2/42.3 kB ? eta -:--:--
   ----------------------------- ---------- 30.7/42.3 kB 435.7 kB/s eta 0:00:01
   ---------------------------------------- 42.3/42.3 kB 410.8 kB/s eta 0:00:00


In [214]:
API_KEY = 'AA553B4C0D1679C81FD4161CF9DE36A3'
SECRET_KEY = 'A9358BEBFAFE9BCB1788DEBD40DF0982'

In [ ]:
import json
import time
import base64
import requests
from deep_translator import GoogleTranslator


class Text2ImageAPI:

    def __init__(self, url, api_key, secret_key):
        self.URL = url
        self.AUTH_HEADERS = {
            'X-Key': f'Key {api_key}',
            'X-Secret': f'Secret {secret_key}',
        }

    def get_model(self):
        response = requests.get(self.URL + 'key/api/v1/models', headers=self.AUTH_HEADERS)
        data = response.json()
        return data[0]['id']

    def generate(self, prompt, model, images=1, width=512, height=512):
        params = {
            "type": "GENERATE",
            "numImages": images,
            "width": width,
            "height": height,
            "generateParams": {
                "query": f"{prompt}"
            }
        }

        data = {
            'model_id': (None, model),
            'params': (None, json.dumps(params), 'application/json')
        }
        response = requests.post(self.URL + 'key/api/v1/text2image/run', headers=self.AUTH_HEADERS, files=data)
        data = response.json()
        return data['uuid']

    def check_generation(self, request_id, attempts=10, delay=10):
        while attempts > 0:
            response = requests.get(self.URL + 'key/api/v1/text2image/status/' + request_id, headers=self.AUTH_HEADERS)
            data = response.json()
            if data['status'] == 'DONE':
                return data['images'][0]  # Вернем только первое изображение, если их несколько

            attempts -= 1
            time.sleep(delay)

    def save_image(self, image_base64, filename="image.png"):
        # Декодируем base64 и сохраняем как файл
        image_data = base64.b64decode(image_base64)
        with open(filename, "wb") as file:
            file.write(image_data)


if __name__ == '__main__':
    api = Text2ImageAPI('https://api-key.fusionbrain.ai/', API_KEY, SECRET_KEY)
    model_id = api.get_model()
    for i, motto in enumerate(mottoes):
        print(i,motto)
        translated = GoogleTranslator(source='ru', target='en').translate(motto)
        uuid = api.generate(f"A detailed and visually compelling image representing the theme of\
        {translated} with intricate textures,\
        no text, and an realistic visual style.", model_id)
        image_base64 = api.check_generation(uuid)
        if image_base64:
            api.save_image(image_base64, f"{i}.png")
            print(f"Image saved as {i}.png")
        else:
            print("Image generation failed or timed out")


0 "Из детства - с любовью к языкам!"
